# **Project 30: Bird Species Predictor**

***numpy*** — *Brings in NumPy to handle fast numerical operations and arrays (the backbone of image and matrix processing).*

***cv2*** — *Imports OpenCV for computer vision tasks like image loading, processing, and object detection.*

***random*** — *Provides functions to generate random numbers or make random selections.*

***warnings*** — *Allows you to control, filter, or suppress warning messages during execution.*

***listdir*** — *Imports listdir to retrieve the list of files and folders inside a directory.*

In [ ]:
import numpy as np
import cv2
import random
import warnings
from os import listdir

**Suppresses all warning messages so they are not displayed during program execution.**

In [ ]:
warnings.filterwarnings("ignore")

**!pip install unrar — Installs the unrar utility so the system can handle .rar compressed files.**

**!unrar x "/content/Bird Species Dataset.rar" — Extracts all contents of the RAR file into the current working directory.**

In [1]:
!pip install unrar
!unrar x "/content/Bird Species Dataset.rar"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/Bird Species Dataset.rar

Creating    Bird Speciees Dataset                                     OK
Creating    Bird Speciees Dataset/AMERICAN GOLDFINCH                  OK
Extracting  Bird Speciees Dataset/AMERICAN GOLDFINCH/001.jpg               0%  OK 
Extracting  Bird Speciees Dataset/AMERICAN GOLDFINCH/002.jpg               0%  OK 
Extracting  Bird Speciees Dataset/AMERICAN GOLDFINCH/003.jpg               0%  OK 
Extracting  Bird Speciees Dataset/AMERICAN GOLDFINCH/004.jpg               0%  OK 
Extracting  Bird Speciees Dataset/AMERICAN GOLDFINCH/005.jpg               0%  OK 
Extracting  Bird Speciees Dataset/AMERICAN GOLDFINCH/006.jpg               0%  OK 
Extracting  Bird Speciees Dataset/AMERICAN GOLDFINCH/007.jpg               0%  OK 
Extracting  Bird Speciees Dataset/AMERICAN GOLDFINCH/008.jpg               0%

* **Sets the dataset directory, initializes image and label containers, and extracts sorted class names from folder structure.**
* **Iterates through each class and its images, loading them from disk while skipping corrupted files.**
* **Resizes all images to 150×150 and converts them to RGB for consistent model input.**
* **Stores processed images and their corresponding class labels into lists.**
* **Converts images and labels into NumPy arrays and normalizes pixel values to 0–1, then reports total images loaded.**


In [6]:
DATA_DIR = "/content/Bird Speciees Dataset"

image_list, label_list = [], []
classes = sorted(listdir(DATA_DIR))

print("Loading images...")
for cls in classes:
    for file in listdir(f"{DATA_DIR}/{cls}"):
        path = f"{DATA_DIR}/{cls}/{file}"
        img = cv2.imread(path)
        if img is None:
            continue
        img = cv2.resize(img, (150, 150))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image_list.append(img)
        label_list.append(cls)

X = np.array(image_list, dtype="float32") / 255.0
y = np.array(label_list)

print(f"✓ Loaded {len(X)} images")

Loading images...
✓ Loaded 811 images


* **Imports TensorFlow as the core deep learning framework.**
* **Imports Keras, TensorFlow’s high-level API for building and training neural networks.**
* **Imports common neural network layers such as Dense, Conv2D, and MaxPooling.**
* **Imports ImageDataGenerator for real-time image augmentation and preprocessing.**


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

* **Imports a function to split data into training and testing sets.**
* **Imports a tool to convert class labels into one-hot encoded format.**
* **Imports a utility to compute class weights for handling imbalanced datasets.**
* **Imports evaluation tools to generate a classification report and confusion matrix.**


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

* **Initializes a LabelBinarizer to encode categorical labels.**
* **Transforms the original class labels into one-hot encoded vectors.**
* **Calculates the total number of unique classes in the dataset.**


In [ ]:
lb = LabelBinarizer()
y_encoded = lb.fit_transform(y)
num_classes = len(lb.classes_)

* **Imports the pickle module for serializing Python objects.**
* **Opens a file in write-binary mode to store the label classes.**
* **Saves the class labels into the file using pickle.**
* **Prints a confirmation message that labels have been saved.**

In [19]:
import pickle

with open("bird_labels.pickle", "wb") as f:
    pickle.dump(lb.classes_, f)

print("✓ Labels saved")

✓ Labels saved


* **Splits the dataset into training and testing sets, keeping 20% for testing and preserving class distribution.**
* **Further splits the training set into training and validation sets, again preserving class balance.**
* **Converts one-hot encoded training labels back to class indices for tasks like class weighting or metrics.**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=np.argmax(y_train, axis=1))

y_train_classes = np.argmax(y_train, axis=1)

* **Computes balanced class weights to handle class imbalance in the training set.**
* **Stores the computed weights in a dictionary mapping class index to weight.**
* **Prints the class weights for each bird species to verify the values.**

In [9]:
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train_classes), y=y_train_classes)

class_weight_dict = {i: w for i, w in enumerate(class_weights)}

print("Class Weights:")
for i, w in class_weight_dict.items():
    print(f"{lb.classes_[i]}: {w:.2f}")

Class Weights:
AMERICAN GOLDFINCH: 0.95
BARN OWL: 1.05
CARMINE BEE-EATER: 1.03
DOWNY WOODPECKER: 0.99
EMPEROR PENGUIN: 0.97
FLAMINGO: 1.02


* **Defines a function to create a convolutional neural network with a specified input shape and number of classes.**
* **Uses multiple Conv2D layers with BatchNormalization, ReLU activation, MaxPooling, and Dropout for feature extraction and regularization.**
* **Applies GlobalAveragePooling before the dense layers to reduce spatial dimensions.**
* **Adds a fully connected Dense layer with L2 regularization, BatchNormalization, and Dropout to prevent overfitting.**
* **Outputs a final Dense layer with softmax activation for multi-class classification and prints the model summary.**


In [10]:
def create_balanced_model(input_shape=(150,150,3), num_classes=6):
    model = keras.Sequential([

        layers.Conv2D(32, 3, padding="same", input_shape=input_shape),
        layers.BatchNormalization(), layers.Activation("relu"), layers.MaxPooling2D(), layers.Dropout(0.3),

        layers.Conv2D(64, 3, padding="same"), layers.BatchNormalization(), layers.Activation("relu"),
        layers.MaxPooling2D(), layers.Dropout(0.3),

        layers.Conv2D(128, 3, padding="same"), layers.BatchNormalization(), layers.Activation("relu"),
        layers.GlobalAveragePooling2D(), layers.Dropout(0.4),

        layers.Dense(128, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.BatchNormalization(), layers.Dropout(0.5),

        layers.Dense(num_classes, activation="softmax")

    ])
    return model

model = create_balanced_model(num_classes=num_classes)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 150, 150, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 150, 150, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 150, 150, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 75, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 75, 75, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 37, 37, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,942 (437.27 KB)

 Trainable params: 111,238 (434.52 KB)

 Non-trainable params: 704 (2.75 KB)

* **Configures the model for training using the Adam optimizer with a learning rate of 0.001.**
* **Sets categorical cross-entropy as the loss function for multi-class classification.**
* **Tracks accuracy as the performance metric during training.**

In [11]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

* **Creates an ImageDataGenerator to apply real-time data augmentation on training images.**
* **Randomly rotates images up to 25 degrees to increase model robustness.**
* **Shifts images horizontally and vertically by up to 15% to simulate different positions.**
* **Applies random zoom up to 20% and horizontal flips for variety.**
* **Uses nearest-neighbor filling for pixels introduced during transformations.**

In [12]:
datagen = ImageDataGenerator(rotation_range=25, width_shift_range=0.15, height_shift_range=0.15, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

* **Sets up callbacks to control training behavior and improve performance.**
* **EarlyStopping monitors validation loss and stops training if it doesn’t improve for 12 epochs, restoring the best weights.**
* **ReduceLROnPlateau reduces the learning rate by half if validation loss plateaus for 5 epochs, down to a minimum of 1e-6.**

In [13]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=12, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6)
]

* **Trains the model using augmented data generated in real-time from the training set.**
* **Sets batch size to 32 and calculates steps per epoch based on training samples.**
* **Runs for up to 80 epochs while validating on the validation set.**
* **Uses previously defined callbacks to stop early or adjust learning rate automatically.**
* **Applies class weights to handle imbalanced classes and prints progress during training.**

In [14]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), steps_per_epoch=len(X_train)//32, epochs=80, validation_data=(X_val, y_val),
      callbacks=callbacks, class_weight=class_weight_dict, verbose=1)

Epoch 1/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 22s 742ms/step - accuracy: 0.2351 - loss: 3.6584 - val_accuracy: 0.2692 - val_loss: 3.0010 - learning_rate: 0.0010
Epoch 2/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2188 - loss: 3.5098 - val_accuracy: 0.3077 - val_loss: 3.0000 - learning_rate: 0.0010
Epoch 3/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 209ms/step - accuracy: 0.4366 - loss: 2.9504 - val_accuracy: 0.1615 - val_loss: 2.9884 - learning_rate: 0.0010
Epoch 4/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4688 - loss: 2.4990 - val_accuracy: 0.1615 - val_loss: 2.9870 - learning_rate: 0.0010
Epoch 5/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 171ms/step - accuracy: 0.5243 - loss: 2.7753 - val_accuracy: 0.1615 - val_loss: 2.9828 - learning_rate: 0.0010
Epoch 6/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4688 - loss: 2.5518 - val_accuracy: 0.1615 - val_loss: 2.9814 - learning_rate: 0.0010
Epoch 7/80
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 167ms/step - accuracy: 0.4670 - loss: 2.6053 - val_

* **Uses the trained model to predict class probabilities for the test set.**
* **Converts predicted probabilities into class indices using argmax.**
* **Converts one-hot encoded true labels into class indices for evaluation.**

In [15]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step


* **Prints a detailed classification report showing precision, recall, F1-score, and support for each class.**
* **Prints the confusion matrix to visualize correct and incorrect predictions across all classes.**

In [16]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=lb.classes_))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Classification Report:
                    precision    recall  f1-score   support

AMERICAN GOLDFINCH       1.00      0.79      0.88        29
          BARN OWL       0.64      0.96      0.77        26
 CARMINE BEE-EATER       0.91      0.77      0.83        26
  DOWNY WOODPECKER       0.90      0.68      0.78        28
   EMPEROR PENGUIN       0.66      0.96      0.78        28
          FLAMINGO       1.00      0.65      0.79        26

          accuracy                           0.80       163
         macro avg       0.85      0.80      0.81       163
      weighted avg       0.85      0.80      0.81       163


Confusion Matrix:
[[23  4  0  1  1  0]
 [ 0 25  0  1  0  0]
 [ 0  3 20  0  3  0]
 [ 0  3  0 19  6  0]
 [ 0  1  0  0 27  0]
 [ 0  3  2  0  4 17]]


* **Saves the trained model to a file named Bird Classifier.keras for later use or deployment.**
* **Prints confirmation that both the model and the previously saved label file are stored successfully.**

In [6]:
model.save("Bird Classifier.keras")
print("\n✓ Bird Classifier.keras saved")
print("✓ bird_labels.pickle saved")


✓ Bird Classifier.keras saved
✓ bird_labels.pickle saved
